# Implementação do algoritimo completo, sem compromisso com  perfomance

## Carregando as matrizes de dissimilaridade

In [12]:
import os
import numpy as np

DATA_BASE_PATH = "./data"

FAC_FILE = os.path.join(DATA_BASE_PATH, "mfeat-fac-dissimilarity.npy")
FOU_FILE = os.path.join(DATA_BASE_PATH, "mfeat-fou-dissimilarity.npy")
KAR_FILE = os.path.join(DATA_BASE_PATH, "mfeat-kar-dissimilarity.npy")

fac_dis = np.load(FAC_FILE)
fou_dis = np.load(FOU_FILE)
kar_dis = np.load(KAR_FILE)


## Definindo a função match

![Match function](./img/match_function.png)

In [13]:
def cluster_matching_function(weight_matrix, 
                              cluster_number,
                              element, 
                              prototypes, 
                              dissimilarity_matrices):
    """
        :params: weight_matrix: numpy array-like 
                    matriz K x P de pesos das matrizes de dissimilaridades por cluster
                cluster_number: int
                    Número do cluster em questão
                element: int
                    Índice do elemento (entre 0 e N-1)
                prototypes: list like
                    Lista de tamanho K dos protótipos de cada cluster
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade
        
        :return: float
                    
    """
    
    # Criando aliases compatíveis com os nomes da fórmula
    k = cluster_number
    D = dissimilarity_matrices
    p = len(D)
    Gk = prototypes[k]

    weights = weight_matrix[k,:p]
    dissimilarities_sum = np.array([sum([dj[element, e] for e in Gk]) for dj in D])
    return np.dot(weights, dissimilarities_sum)



## Definindo a função objetivo

![Objetive function](./img/objective_function.png)

In [14]:
import functools
import operator
import math 

def objective_function(clusters_qtd, 
                       elements_qtd, 
                       adequacy_criterion, 
                       m, 
                       weight_matrix, 
                       prototypes,
                       dissimilarity_matrices):
        """
        :params: clusters_qtd: int
                    Quantidade total de clusters
                elements_qtd: int 
                    Quantidade de elementos da base de dados
                adequacy_criterion: numpy array-like
                    Matriz u de tamanho N x K contendo a índice de adequação de cada elemente a cada cluster
                m: int
                    Fator de ponderação do índice de adequação
                weight_matrix: 
                     matriz K x P de pesos das matrizes de dissimilaridades por cluster
                prototypes: list like
                    Lista de tamanho K dos protótipos de cada cluster
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade
                    
        :return: float
                    
    """
        
    u = np.power(adequacy_criterion, m) # Resolvendo a exponeciação de u de uma vez só
    l = weight_matrix
    D = dissimilarity_matrices
    K = clusters_qtd
    Gk = prototypes
    N = elements_qtd
    match = cluster_matching_function # Criando um alias para reduzir o nome da função de matching
  
    J = [sum([u[i, k] * match(l, k, i, Gk, D) for i in range(N)]) 
          for k in range(K)]


    return sum(J) 
    

## Definindo a função da cálculo de protótipos

![Prototype function](./img/prototype_function.png)

In [19]:
def get_prototypes(elements_qtd, 
                  q, 
                  m, 
                  s,
                  cluster_number,
                  adequacy_criterion,
                  dissimilarity_matrices, 
                  weight_matrix):
    
    """
        :params:
                elements_qtd: int 
                    Quantidade de elementos da base de dados
                q: int
                    Quantidade de elementos protótipos
                m: int
                    Fator de ponderação do índice de adequação
                s: int
                    Fator de ponderação dos pesos das matrizes
                cluster_number: int
                    Quantidade total de clusters
                adequacy_criterion: numpy array-like
                    Matriz u de tamanho N x K contendo a índice de adequação de cada elemente a cada cluster
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade
                weight_matrix: 
                     matriz K x P de pesos das matrizes de dissimilaridades por cluster
                    
        :return: list
                    
    """
    
    G = []
    k = cluster_number,
    D = dissimilarity_matrices
    u = np.power(adequacy_criterion, m) 
    l = np.power(weight_matrix, s)
    N = elements_qtd
    p = len(D)

    def dist(element):
        """
            Função auxiliar para cálculo da distância de um elemento qualquer 
            em relação a todos os outros da base de dados, consideran as matrizes de dissimilaridade e 
            o critério de adequação
            
            return: (int, float)
                (lement, soma das distâncias)
        """
        return element, sum([u[i,k] * sum([l[k,j] * D[j][element, i] for j in range(p)])
              for i in range(elements_qtd)

              ]
            )


    while (len(G) < q):
        #Calculando todas as distâncias dos elementos que ainda não estão em G
        all_distances = (dist(i) for i in range(N) if i not in G) 
        element, _ = min(all_distances, key=functools.itemgetter(1))
        G.append(element)

    return G

## Definindo a função de computo da matriz de relevâcia

![Funções de peso](./img/vector_weights_function.png)

In [ ]:
def compute_relevance_weights(clusters_qtd,
                              dissimilarity_matrices,
                              prototypes,
                              elements_qtd,
                              adequacy_criterion,
                              m):
  
    D = dissimilarity_matrices
    p = len(D)
    Gk = prototypes
    K = clusters_qtd
    n = elements_qtd
    u = np.power(adequacy_criterion, m)
    match = cluster_matching_function
    l = np.zeros(K, p)

    for k in range(K):
        for j in range(p):
        weight_dissimilarity_sum = [sum([u[i,k] * match(l, k, i, Gk, D[h]) for i in range(n)])
                                    for h in range(p)]

        weight_dissimilarity_prod = functools.reduce(operator.mul, weight_dissimilarity_sum)
        weight_dissimilarity_sum = sum([u[i,k] * match(l, k, i, Gk, D[j]) for i in range(n)])
        l[k,j] = math.pow(weight_dissimilarity_prod, 1/p) / weight_dissimilarity_sum

    return l